In [1]:
# ============================================================
# 📦 SETUP
# ============================================================
!git clone https://github.com/NVlabs/edm2.git /kaggle/working/edm2
%cd /kaggle/working/edm2
!pip install click tqdm psutil scipy pillow matplotlib --quiet


Cloning into '/kaggle/working/edm2'...
remote: Enumerating objects: 60, done.
remote: Counting objects: 100% (27/27), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 60 (delta 13), reused 10 (delta 10), pack-reused 33 (from 1)
Receiving objects: 100% (60/60), 1.27 MiB | 9.58 MiB/s, done.
Resolving deltas: 100% (24/24), done.
/kaggle/working/edm2


In [2]:
# ============================================================
# 📦 SETUP
# ============================================================

!pip install pillow pandas numpy torch --quiet

import os, math, pickle, torch
import numpy as np
import pandas as pd
from PIL import Image, ImageDraw, ImageFont
from pathlib import Path

import sys
sys.path.append("/kaggle/working/edm2")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 98.7 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 90.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 32.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 7.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 14.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 65.4 MB/s eta 0:00:00:00:0100:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed

In [3]:
# ============================================================
# 🧩 LOAD LANDMARK TEST DATA
# ============================================================

LANDMARK_CSV = "/kaggle/input/landmark-for/celeba64_landmarks_transformed.csv"
TEST_DIR     = "/kaggle/input/celeva-64x64-dataset/celeba64/test"

df = pd.read_csv(LANDMARK_CSV)

img_col = "image_id"
lm_cols = df.columns[1:]   # 10 landmark coords

# Keep only test images
test_files = sorted(os.listdir(TEST_DIR))
df_test = df[df[img_col].isin(test_files)].reset_index(drop=True)

landmarks = df_test[lm_cols].values.astype(np.float32)
filenames = df_test[img_col].values

print("Test samples:", len(df_test))
print("Landmark shape:", landmarks.shape)


Test samples: 19962
Landmark shape: (19962, 10)


In [4]:
# ============================================================
# 🧠 SELECT 3 LANDMARK EXPERIMENTS
# ============================================================

# Landmark indices
LE_X, LE_Y = 0, 1
RE_X, RE_Y = 2, 3

# Inter-eye distance
eye_dist = np.sqrt(
    (landmarks[:, LE_X] - landmarks[:, RE_X])**2 +
    (landmarks[:, LE_Y] - landmarks[:, RE_Y])**2
)

# Horizontal eye alignment (|y_left - y_right|)
eye_y_diff = np.abs(landmarks[:, LE_Y] - landmarks[:, RE_Y])

idx_max_face = np.argmax(eye_dist)
idx_min_face = np.argmin(eye_dist)
idx_symmetric = np.argmin(eye_y_diff)

EXPERIMENTS = {
    "max_face": idx_max_face,
    "min_face": idx_min_face,
    "symmetric_face": idx_symmetric,
}


In [5]:
# ============================================================
# 🧾 PRINT LANDMARKS FOR EACH EXPERIMENT
# ============================================================

for name, idx in EXPERIMENTS.items():
    print("\n" + "="*60)
    print(name.upper())
    print("="*60)
    print("Image:", filenames[idx])
    for c, v in zip(lm_cols, landmarks[idx]):
        print(f"{c:15s}: {v:.2f}")



MAX_FACE
Image: 198369.jpg
lefteye_x      : 23.37
lefteye_y      : 35.96
righteye_x     : 42.07
righteye_y     : 31.64
nose_x         : 35.96
nose_y         : 44.22
leftmouth_x    : 23.73
leftmouth_y    : 45.66
rightmouth_x   : 36.31
rightmouth_y   : 44.94

MIN_FACE
Image: 193355.jpg
lefteye_x      : 30.92
lefteye_y      : 30.20
righteye_x     : 32.72
righteye_y     : 29.84
nose_x         : 23.01
nose_y         : 41.35
leftmouth_x    : 30.56
leftmouth_y    : 52.85
rightmouth_x   : 32.72
rightmouth_y   : 52.85

SYMMETRIC_FACE
Image: 182643.jpg
lefteye_x      : 25.17
lefteye_y      : 32.72
righteye_x     : 38.47
righteye_y     : 32.72
nose_x         : 32.36
nose_y         : 40.63
leftmouth_x    : 26.25
leftmouth_y    : 47.46
rightmouth_x   : 37.39
rightmouth_y   : 47.82


In [6]:
# ============================================================
# 📦 LOAD LANDMARK-CONDITIONAL MODEL
# ============================================================

MODEL_PKL = "/kaggle/input/network-snapshot-0001572-0-100-landmark-kr/pytorch/default/1/network-snapshot-0001572-0.100_landmark_kr.pkl"

device = torch.device("cuda")

with open(MODEL_PKL, "rb") as f:
    net = pickle.load(f)["ema"].to(device).eval()

print("✅ Landmark-conditional model loaded")
print("Condition dim:", net.label_dim)


✅ Landmark-conditional model loaded
Condition dim: 10


In [7]:
# ============================================================
# 🧪 EDM SAMPLER WITH STEP SAVING (KARRAS)
# ============================================================

def edm_sampler_with_steps(
    net, noise, labels, save_dir,
    num_steps=32, sigma_min=0.002, sigma_max=80, rho=7
):
    os.makedirs(save_dir, exist_ok=True)
    device = noise.device

    i = torch.arange(num_steps, device=device)
    sigmas = (
        sigma_max ** (1 / rho)
        + i / (num_steps - 1)
        * (sigma_min ** (1 / rho) - sigma_max ** (1 / rho))
    ) ** rho

    sigmas = torch.cat([sigmas, torch.zeros_like(sigmas[:1])])

    def denoise(x, t):
        return net(x, t, labels)

    x = noise * sigmas[0]

    for i in range(num_steps):
        d = (x - denoise(x, sigmas[i])) / sigmas[i]
        x = x + (sigmas[i+1] - sigmas[i]) * d

        img = x[0].detach().cpu().clamp(-1,1)
        img = ((img+1)/2*255).permute(1,2,0).numpy().astype(np.uint8)
        Image.fromarray(img).save(f"{save_dir}/step_{i:03d}.png")

    return sigmas


In [8]:
# ============================================================
# 🚀 RUN 3 LANDMARK CONDITIONAL TRAJECTORIES
# ============================================================

C, H, W = net.img_channels, net.img_resolution, net.img_resolution

for name, idx in EXPERIMENTS.items():
    print(f"\n🚀 Running {name}")

    noise = torch.randn(1, C, H, W, device=device)
    label = torch.tensor(landmarks[idx], device=device).unsqueeze(0)

    outdir = f"/kaggle/working/landmark_{name}_steps"

    sigmas = edm_sampler_with_steps(
        net=net,
        noise=noise,
        labels=label,
        save_dir=outdir,
        num_steps=32,
    )



🚀 Running max_face

🚀 Running min_face

🚀 Running symmetric_face


In [9]:
# ============================================================
# 🖼️ SAVE 4×8 GRID WITH σ LABELS
# ============================================================

def save_grid(step_dir, sigmas, out_path):
    imgs = [Image.open(f"{step_dir}/step_{i:03d}.png") for i in range(32)]
    w, h = imgs[0].size

    COLS, ROWS = 8, 4
    TOP = 32
    GAP = 18

    canvas = Image.new("RGB", (COLS*w, ROWS*(h+TOP+GAP)), "white")
    draw = ImageDraw.Draw(canvas)

    try:
        font = ImageFont.truetype("DejaVuSans.ttf", 13)
    except:
        font = ImageFont.load_default()

    for i, img in enumerate(imgs):
        r, c = divmod(i, COLS)
        x = c*w
        y = r*(h+TOP+GAP)

        draw.text((x+w//2, y+10), f"σ={sigmas[i]:.2f}", anchor="mm", font=font)
        canvas.paste(img, (x, y+TOP))

    canvas.save(out_path)
    print("✅ Saved:", out_path)

for name in EXPERIMENTS:
    save_grid(
        f"/kaggle/working/landmark_{name}_steps",
        sigmas,
        f"/kaggle/working/landmark_{name}_grid.png"
    )


✅ Saved: /kaggle/working/landmark_max_face_grid.png
✅ Saved: /kaggle/working/landmark_min_face_grid.png
✅ Saved: /kaggle/working/landmark_symmetric_face_grid.png
